In [ ]:
#A = gov
#B = anmawon
#total = union(A,B)
#safe_shop = intersection(total, B)

In [ ]:
path_gov_shop = "manual_edit_government_REgeolocated.csv"
path_anmawon_union = "manual_edit_anmawon_REgeolocated.csv"

import pandas as pd


In [ ]:

gov_shop = pd.read_csv(path_gov_shop)
anmawon_union_shop = pd.read_csv(path_anmawon_union)

gov_shop.rename(columns={"사업장명":"name", "주소": "addr", "전화": "phone"}, inplace=True)

del anmawon_union_shop["지역"]
anmawon_union_shop["checked"] = True

anmawon_union_shop.rename(columns={"사업장명":"name", "주소": "addr", "전화번호": "phone", "링크": "hlink"}, inplace=True)
for index, row in anmawon_union_shop.iterrows():
    new_link = "http://www.anmawon.com/FindShop/Detail?shopId=" + str(row.loc["hlink"][24:])  # replace /FindShop/Detail?shopId=
    anmawon_union_shop.at[index, "hlink"] = new_link

unique_addr_list = list(set(anmawon_union_shop.loc[:, "NAVER_addr"]).union(set(gov_shop.loc[:, "NAVER_addr"])))


In [ ]:
final_data_list = []
for unique_addr in unique_addr_list:
    if (gov_shop['NAVER_addr'].eq(unique_addr)).any():
        row = gov_shop.loc[gov_shop['NAVER_addr'] == unique_addr].to_dict(orient="records")[0]
        name, phone, x, y = [row.get(key) for key in ["name", "phone", "x", "y"]]
        # check union(gov, anmawon)
        if (anmawon_union_shop['NAVER_addr'].eq(unique_addr)).any():
            row = anmawon_union_shop.loc[anmawon_union_shop['NAVER_addr'] == unique_addr].to_dict(orient="records")[0]
            hlink, checked = row["hlink"], "true"
        else:
            hlink, checked = "", "false"
    else:  # only in anmawon_union_shop
        row = anmawon_union_shop.loc[anmawon_union_shop['NAVER_addr'] == unique_addr].to_dict(orient="records")[0]
        name, phone, x, y = [row.get(key) for key in ["name", "phone", "x", "y"]]
        hlink, checked = row["hlink"], "unknown"

    final_data_list.append(
        {'unique_addr': unique_addr,
        'name': name,
        'phone':phone,
        'x':x ,
        'y':y ,
        'hlink': hlink ,
        'checked':checked })

final_data = pd.DataFrame({c: pd.Series(dtype=t) for c, t in {'unique_addr': 'str', 'name': 'str', 'phone': 'str', 'x':"float", 'y':'float', 'checked':'str', 'hlink':'str'}.items()})

final_data = final_data.append(final_data_list, ignore_index=False, sort=False)


In [ ]:
with open("final_data.json", "w", encoding="utf-8") as f:
    final_data.to_json(f, orient="records", force_ascii=False)

    # then, manually change to .js format as "var data = [....]"